In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.4/778.4 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import cv2
import numpy as np
from ultralytics import YOLO, solutions
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

# Initialize the model and video capture
model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture("/content/drive/MyDrive/yolo/race.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

line_points = [(20, 400), (1080, 400)]  # line or region points
classes_to_count = [0, 2, 17]  # person, car, and horse classes for count

# Video writer
video_writer = cv2.VideoWriter("object_counting_output_final.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=line_points,
    classes_names=model.names,
    draw_tracks=True,
    line_thickness=2,
)

# Initialize counters
in_count = 0
out_count = 0
class_counts = {class_id: 0 for class_id in classes_to_count}  # Initialize class counts
unique_ids_in = {class_id: set() for class_id in classes_to_count}  # Initialize unique ID sets for IN
unique_ids_out = {class_id: set() for class_id in classes_to_count}  # Initialize unique ID sets for OUT

def draw_bar_graph(in_count, out_count, class_counts):
    # Create a new figure
    fig, ax = plt.subplots(figsize=(4, 4), dpi=100)

    labels = ['In', 'Out'] + [model.names[class_id] for class_id in classes_to_count]
    counts = [in_count, out_count] + [class_counts[class_id] for class_id in classes_to_count]

    bars = ax.bar(labels, counts, color=['blue', 'orange'] + ['green'] * len(classes_to_count))

    # Annotate bars with count values
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom', color='red')

    ax.set_ylim(0, max(counts) + 5)  # Dynamically adjust y-axis limit

    # Render the plot to a canvas
    canvas = FigureCanvas(fig)
    canvas.draw()

    # Convert canvas to image
    img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    img = img.reshape(canvas.get_width_height()[::-1] + (3,))

    # Release the plot
    plt.close(fig)

    return img

def overlay_class_counts(frame, class_counts):
    y_offset = 50
    x_position = w - 250
    for class_id, count in class_counts.items():
        class_name = model.names[class_id]
        text = f"{class_name}: {count}"
        cv2.putText(frame, text, (x_position, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)  # Red color text
        y_offset += 30

def is_crossing(line_points, point):
    # Check if a point crosses a line
    x1, y1 = line_points[0]
    x2, y2 = line_points[1]
    x, y = point
    if (y1 <= y <= y2) or (y2 <= y <= y1):
        m = (y2 - y1) / (x2 - x1)  # slope of the line
        crossing_x = (y - y1) / m + x1  # x-coordinate of the crossing point
        return x <= crossing_x <= x2
    return False

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = model.track(im0, persist=True, show=False, classes=classes_to_count)

    # Accessing tracking results properly
    for result in results:
        if hasattr(result, 'boxes') and result.boxes is not None:
            for box in result.boxes.data:
                cls_id = int(box[5])
                track_id = int(box[4])
                if cls_id in classes_to_count:
                    if is_crossing(line_points, (box[0], box[1])):
                        if track_id not in unique_ids_in[cls_id]:
                            unique_ids_in[cls_id].add(track_id)
                            class_counts[cls_id] += 1
                    else:
                        if track_id not in unique_ids_out[cls_id]:
                            unique_ids_out[cls_id].add(track_id)
                            class_counts[cls_id] -= 1

    im0 = counter.start_counting(im0, results)

    # Update counts (replace with actual counting logic)
    in_count += np.random.randint(0, 2)
    out_count += np.random.randint(0, 2)

    # Draw bar graph and overlay on the frame
    bar_graph_img = draw_bar_graph(in_count, out_count, class_counts)
    bar_graph_img = cv2.cvtColor(bar_graph_img, cv2.COLOR_RGB2BGR)

    # Resize bar graph image to fit in the video frame
    bar_graph_h, bar_graph_w, _ = bar_graph_img.shape
    bar_graph_x, bar_graph_y = 10, 10
    im0[bar_graph_y:bar_graph_y + bar_graph_h, bar_graph_x:bar_graph_x + bar_graph_w] = bar_graph_img

    # Overlay class counts on the frame
    overlay_class_counts(im0, class_counts)

    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.

0: 384x640 3 horses, 72.3ms
Speed: 2.1ms preprocess, 72.3ms inference, 8.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 horses, 29.4ms
Speed: 4.0ms preprocess, 29.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 horses, 11.2ms
Speed: 8.3ms preprocess, 11.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 horses, 8.5ms
Speed: 4.9ms preprocess, 8.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 horses, 13.1ms
Speed: 1.8ms preprocess, 13.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 horses, 10.1ms
Speed: 2.5ms preprocess, 10.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 horses, 8.5ms
Speed: 1.8ms preprocess, 8.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384,

KeyboardInterrupt: 